In [ ]:
from mlrepricer import listener, redisdb, updateprice, minmax, asintosku, helper

In [ ]:
#redisdb.server_start()

In [ ]:
asintosku.AsintoSku().start()

# listener demon
to get messages from mws queues and dump them into a filesystem

In [ ]:
listener.Listener().start()

In [ ]:
updateprice.Updater().start()

In [ ]:
def get_report(report_name):
    """Call reports easy."""
    report_api = mws.apis.Reports(**helper.mwscred)
    request_response = report_api.request_report(report_name)
    report_id = helper.auto_get_report_id(request_response)
    if report_id:
        report = report_api.get_report(report_id).parsed
    return report

def read_filter(report):
    i = pd.read_csv(io.StringIO(report), sep='\t', decimal='.',
                    thousands=None, dayfirst=True)
    i = i[i.loc[:, 'item-condition'] == 11]
    d = i.loc[:, ['seller-sku', 'asin1', 'fulfillment-channel']]
    d['county'] = 1
    return d.groupby(['asin1', 'fulfillment-channel']).agg(
        {'seller-sku': 'first', 'county': 'count'})


def deduplicate(listingperasin):
    """Remove duplicates."""
    duplicates = listingperasin[listingperasin.loc[:, 'county'] > 1]
    if len(duplicates) >= 1:
        print(f'you may want to delete one listing per asin {duplicates}')

    # those we return
    return listingperasin[listingperasin.county == 1].reset_index()


def reorganize(nocollusion):
    """We do some final cleanup and reorganize columns."""
    # When fulfillment-channel is DEFAULT its seller fulfilled
    nocollusion['isprime'] = np.where(
        nocollusion['fulfillment-channel'] == 'DEFAULT', 0, 1)
    nocollusion.drop(['fulfillment-channel', 'county'], axis=1, inplace=True)
    nocollusion.rename(
        {'asin1': 'asin', 'seller-sku': 'seller_sku'}, axis=1, inplace=True)
    helper.dump_dataframe(nocollusion, 'asintosku')


In [ ]:
import io
import mws
from mws.apis.reports import ReportType
import numpy as np
import pandas as pd
import threading

report = get_report(ReportType.ACTIVE_LISTINGS.value)

In [ ]:
# store the report
reorganize(deduplicate(read_filter(report)))

In [ ]:
minmax.dump_csv()
u = minmax.load_csv()
u[u.asin == 'B07H9WFDSR_seller']

In [ ]:
helper.load_dataframe('asintosku').reset_index()